In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [37]:
path = (r'C:\\Users\\ajifo\\Machine learning project\\Walmart sales forecast\\data')

In [38]:
train_data = pd.read_csv (f'{path}\\train.csv')
test_data = pd.read_csv (f'{path}\\test.csv')
features_data = pd.read_csv (f'{path}\\features.csv')
stores_data = pd.read_csv (f'{path}\\stores.csv')

Merging Everything into one table

In [39]:
train_data = train_data.merge(features_data.drop(columns='IsHoliday'), on=["Store", "Date"], how="left")
train_data = train_data.merge(stores_data, on="Store", how="left")

test_data = test_data.merge(features_data.drop(columns='IsHoliday'), on=["Store", "Date"], how="left")
test_data = test_data.merge(stores_data, on="Store", how="left")

In [40]:
display(train_data.shape)
display(train_data.head())
display(train_data.info())
display(train_data.describe())


(421570, 16)

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
0,1,1,2010-02-05,24924.50,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
1,1,1,2010-02-12,46039.49,True,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,A,151315
2,1,1,2010-02-19,41595.55,False,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,A,151315
3,1,1,2010-02-26,19403.54,False,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,A,151315
4,1,1,2010-03-05,21827.90,False,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,A,151315


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         421570 non-null  int64  
 1   Dept          421570 non-null  int64  
 2   Date          421570 non-null  object 
 3   Weekly_Sales  421570 non-null  float64
 4   IsHoliday     421570 non-null  bool   
 5   Temperature   421570 non-null  float64
 6   Fuel_Price    421570 non-null  float64
 7   MarkDown1     150681 non-null  float64
 8   MarkDown2     111248 non-null  float64
 9   MarkDown3     137091 non-null  float64
 10  MarkDown4     134967 non-null  float64
 11  MarkDown5     151432 non-null  float64
 12  CPI           421570 non-null  float64
 13  Unemployment  421570 non-null  float64
 14  Type          421570 non-null  object 
 15  Size          421570 non-null  int64  
dtypes: bool(1), float64(10), int64(3), object(2)
memory usage: 48.6+ MB


None

,Store,Dept,Weekly_Sales,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Size
count,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,150681.000000,111248.000000,137091.000000,134967.000000,151432.000000,421570.000000,421570.000000,421570.000000
mean,22.200546,44.260317,15981.258123,60.090059,3.361027,7246.420196,3334.628621,1439.421384,3383.168256,4628.975079,171.201947,7.960289,136727.915739
std,12.785297,30.492054,22711.183519,18.447931,0.458515,8291.221345,9475.357325,9623.078290,6292.384031,5962.887455,39.159276,1.863296,60980.583328
min,1.000000,1.000000,-4988.940000,-2.060000,2.472000,0.270000,-265.760000,-29.100000,0.220000,135.160000,126.064000,3.879000,34875.000000
25%,11.000000,18.000000,2079.650000,46.680000,2.933000,2240.270000,41.600000,5.080000,504.220000,1878.440000,132.022667,6.891000,93638.000000
50%,22.000000,37.000000,7612.030000,62.090000,3.452000,5347.450000,192.000000,24.600000,1481.310000,3359.450000,182.318780,7.866000,140167.000000
75%,33.000000,74.000000,20205.852500,74.280000,3.738000,9210.900000,1926.940000,103.990000,3595.040000,5563.800000,212.416993,8.572000,202505.000000
max,45.000000,99.000000,693099.360000,100.140000,4.468000,88646.760000,104519.540000,141630.610000,67474.850000,108519.280000,227.232807,14.313000,219622.000000


In [41]:
# Converting Date to time features
train_data["Date"] = pd.to_datetime(train_data["Date"])
test_data["Date"] = pd.to_datetime(test_data["Date"])

for df in [train_data, test_data]:
    df["Year"] = df["Date"].dt.year
    df["Month"] = df["Date"].dt.month
    df["Week"] = df["Date"].dt.isocalendar().week.astype(int)
    
# Converting bool to string
bool_cols = df.select_dtypes(include=['bool']).columns.tolist()

train_data[bool_cols] = train_data[bool_cols].astype(str)
test_data[bool_cols] = test_data[bool_cols].astype(str)

In [42]:
train_data.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size,Year,Month,Week
0,1,1,2010-02-05,24924.50,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315,2010,2,5
1,1,1,2010-02-12,46039.49,True,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,A,151315,2010,2,6
2,1,1,2010-02-19,41595.55,False,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,A,151315,2010,2,7
3,1,1,2010-02-26,19403.54,False,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,A,151315,2010,2,8
4,1,1,2010-03-05,21827.90,False,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,A,151315,2010,3,9


In [43]:
missing_values = train_data.isnull().sum()
Zero = (train_data==0).sum()

pd.DataFrame({'Missing Values': missing_values, 'Zero Values': Zero})

,Missing Values,Zero Values
Store,0,0
Dept,0,0
Date,0,0
Weekly_Sales,0,73
IsHoliday,0,0
Temperature,0,0
Fuel_Price,0,0
MarkDown1,270889,0
MarkDown2,310322,207
MarkDown3,284479,67


In [44]:
missing_values = test_data.isnull().sum()
Zero = (test_data==0).sum()

pd.DataFrame({'Missing Values': missing_values, 'Zero Values': Zero})

,Missing Values,Zero Values
Store,0,0
Dept,0,0
Date,0,0
IsHoliday,0,0
Temperature,0,0
Fuel_Price,0,0
MarkDown1,149,0
MarkDown2,28627,0
MarkDown3,9829,0
MarkDown4,12888,0


In [45]:
#dropping the markdown columns
cols = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']

train_data.drop(columns=cols, axis=1, inplace=True)
test_data.drop(columns=cols, axis=1, inplace=True)

#fill zeros with nan
train_data['Weekly_Sales'] = train_data['Weekly_Sales'].replace(0, np.nan)
train_data.dropna(subset=['Weekly_Sales'], inplace=True)

#Fill nan inside Test data with 0
test_data.fillna(0, inplace=True)
    

In [46]:
display(train_data.duplicated().sum())
display(test_data.duplicated().sum())

0

0

In [47]:
display(train_data['Type'].unique())

array(['A', 'B', 'C'], dtype=object)

In [48]:
# Finding the largest store between the three Type of stores
largest_store = train_data.loc[train_data['Size'].idxmax()]
largest_store

Store                            13
Dept                              1
Date            2010-02-05 00:00:00
Weekly_Sales                46761.9
IsHoliday                     False
Temperature                   31.53
Fuel_Price                    2.666
CPI                      126.442065
Unemployment                  8.316
Type                              A
Size                         219622
Year                           2010
Month                             2
Week                              5
Name: 117606, dtype: object

In [49]:
# Summary of the store Weekly sales
summary = train_data.groupby('Type')['Weekly_Sales'].agg(['sum', 'mean', 'max'])
display(summary)

,sum,mean,max
Type,,,
A,4.331015e+09,20102.553414,474330.10
B,2.000701e+09,12239.471543,693099.36
C,4.055035e+08,9521.544274,112152.35


Store A has the highest Average weekly sales and the highest total weekly sales. While store B has the maximum weekly sale. This shows that Store A consistently perform the best.

In [50]:
# train_data.to_csv('Clean_train_data.csv', index=False)

In [51]:
cat_cols = train_data.select_dtypes(include=['object']).columns.tolist()

# Initialize encoder dictionary
encoders = {}

for col in cat_cols:
    # Create a LabelEncoder per column
    le = LabelEncoder()
    
    # Fit on combined unique values (train + test)
    combined_values = pd.concat([train_data[col], test_data[col]], axis=0).astype(str)
    le.fit(combined_values)
    
    # Transform train and test
    train_data[col] = le.transform(train_data[col].astype(str))
    test_data[col] = le.transform(test_data[col].astype(str))
    
    # Save encoder if needed later (for new data)
    encoders[col] = le

In [52]:
train_data.columns

Index(['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday', 'Temperature',
       'Fuel_Price', 'CPI', 'Unemployment', 'Type', 'Size', 'Year', 'Month',
       'Week'],
      dtype='object')

In [53]:
Features = ['Store', 'Dept', 'IsHoliday', 'Temperature',
       'Fuel_Price', 'CPI', 'Unemployment', 'Type', 'Size', 'Year', 'Month',
       'Week']

target = ['Weekly_Sales']

In [54]:
X = train_data[Features]
y = train_data[target]

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [56]:
len(X_train), len(y_train), len(X_test), len(y_test)

(337197, 337197, 84300, 84300)

In [57]:
y_mean = y_test.mean()
y_pred_baseline = [y_mean] * len(y_test)

In [58]:
rf = RandomForestRegressor(
    n_estimators=300,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42,
    n_jobs=-1
)

In [59]:
rf.fit(X_train, y_train.values.ravel())

RandomForestRegressor(n_estimators=300, n_jobs=-1, random_state=42)

In [61]:
rf.score(X_train, y_train.values.ravel())

0.9967002703548743

In [62]:
y_pred = rf.predict(X_test)

In [63]:
rf_mae = mean_absolute_error(y_test, y_pred)
display(rf_mae)
base_mae = mean_absolute_error(y_test, y_pred_baseline)
display(base_mae)
r2 = r2_score(y_test, y_pred)
display(r2)

1412.1885350925272

15097.433062792079

0.9749905625263827

In [64]:
rf_mse = mean_squared_error(y_test, y_pred)
display(rf_mse)
base_mse = mean_squared_error(y_test, y_pred_baseline)
display(base_mse)

12820990.212456655

512646085.14212495

In [65]:
X_train_final = train_data[Features]
y_train_final = train_data[target]
X_test_final = test_data[Features]

In [66]:
rf.fit(X_train_final, y_train_final.values.ravel())

RandomForestRegressor(n_estimators=300, n_jobs=-1, random_state=42)

In [67]:
rf.score(X_train_final, y_train_final.values.ravel())

0.9969786694851384

In [68]:
test_data['Weekly_Sales_Predicted'] = rf.predict(X_test_final)

In [70]:
submission = test_data[['Store', 'Dept', 'Date', 'IsHoliday', 'Weekly_Sales_Predicted']]
submission.to_csv(r'C:\\Users\\ajifo\\Machine learning project\\Walmart sales forecast\\data\\submission.csv', index=False)

In [72]:
import joblib
joblib.dump(rf, r'C:\\Users\\ajifo\\Machine learning project\\Walmart sales forecast\\model\\random_forest_model.pkl')

['C:\\\\Users\\\\ajifo\\\\Machine learning project\\\\Walmart sales forecast\\\\model\\\\random_forest_model.pkl']